##Importing Libararies

In [ ]:
import cv2
import os
import numpy as np
import PIL as Image
import matplotlib
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
plt.style.use('ggplot')
import warnings
warnings.filterwarnings('ignore')

##Preparing Dataset

In [ ]:
highres_images_dir = '/content/drive/MyDrive/Project/GANs/Sandstone_high_low_res/higres'
lowres_images_dir = '/content/drive/MyDrive/Project/GANs/Sandstone_high_low_res/lowres'

In [ ]:
def get_data(highres_images_dir, lowres_images_dir):
  highres_images_path_list = [os.path.join(highres_images_dir, x) for x in sorted(os.listdir(highres_images_dir))]
  lowres_images_path_list = [os.path.join(lowres_images_dir, x) for x in sorted(os.listdir(lowres_images_dir))]
  return highres_images_path_list, lowres_images_path_list

In [ ]:
highres_images_path_list, lowres_images_path_list = get_data(highres_images_dir, lowres_images_dir)

In [ ]:
print(highres_images_path_list)
print(f'Size of Dataset: ', len(highres_images_path_list))

In [ ]:
print(lowres_images_path_list)
print(f'Size of Dataset: ', len(highres_images_path_list))

##Visualizing Dataset

In [ ]:
def show_image(image, title='', cmap = None):
  plt.imshow(image, cmap=cmap, alpha=1)
  if title is not None:
    plt.title(title)
  plt.axis('off')

In [ ]:
lowres_image = cv2.imread(lowres_images_path_list[5])
show_image(lowres_image, 'Low Quality Image')

In [ ]:
print(f'Shape Of Low Resolution Image:',lowres_image.shape)

In [ ]:
highres_image = cv2.imread(highres_images_path_list[5])
show_image(highres_image, 'High Quality Image')

In [ ]:
print(f'Shape Of High Resolution Image:',highres_image.shape)

In [ ]:
def display_dataset(highres_images_path_list, lowres_images_path_list):
  plt.figure(figsize=(10,9))
  id = 0
  for i in tqdm(range(12)):
    plt.subplot(3,4,i+1)
    if i%2==0:
      highres_image_path = highres_images_path_list[id]
      highres_image = cv2.imread(highres_image_path)
      show_image(highres_image, cmap='gray')

    elif i%2==1:
      lowres_image_path = lowres_images_path_list[id]
      lowres_image = cv2.imread(lowres_image_path)
      show_image(lowres_image, cmap='gray')      
      id = id + 1
  plt.tight_layout()
  plt.suptitle('Dataset', size=16)
  plt.show()

In [ ]:
display_dataset(highres_images_path_list, lowres_images_path_list)

##Loading Data

In [ ]:
def build_data(lowres_images_path_list, highres_images_path_list):
  X = np.zeros((len(lowres_images_path_list), 64, 64, 3), dtype=np.uint8).astype(np.float32)
  y = np.zeros((len(highres_images_path_list), 256, 256, 3), dtype=np.uint8).astype(np.float32)

  for i, (lowres_image_path, highres_image_path) in tqdm(enumerate(zip(lowres_images_path_list, highres_images_path_list))):
    lowres_img = cv2.imread(lowres_image_path)
    lowres_img = lowres_img/255.
    X[i] = lowres_img

    highres_img = cv2.imread(highres_image_path)
    highres_img = highres_img/255.
    y[i] = highres_img

  return X, y  

In [ ]:
X, y = build_data(lowres_images_path_list, highres_images_path_list)

In [ ]:
print('Input Data Shape & Size:',X.shape)

In [ ]:
print('Input Data Shape & Size:',y.shape)

##Importing Dependencies

In [ ]:
import numpy as np
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, PReLU,BatchNormalization, Flatten
from tensorflow.keras.layers import UpSampling2D, LeakyReLU, Dense, Input, add
from tensorflow.keras.applications import VGG19
from tensorflow.keras.utils import plot_model

##Building Attention U-Net Model

In [ ]:
def resnet_block(ip):   
  res_model = Conv2D(64, (3,3), padding = "same")(ip)
  res_model = BatchNormalization(momentum = 0.5)(res_model)
  res_model = PReLU(shared_axes = [1,2])(res_model)
  
  res_model = Conv2D(64, (3,3), padding = "same")(res_model)
  res_model = BatchNormalization(momentum = 0.5)(res_model)
  
  return add([ip,res_model])

In [ ]:
def upscale_block(ip): 
  up_model = Conv2D(256, (3,3), padding="same")(ip)
  up_model = UpSampling2D(size = 2 )(up_model)
  up_model = PReLU(shared_axes=[1,2])(up_model)
  
  return up_model

In [ ]:
def create_generator(gen_ip, num_res_block):
  layers = Conv2D(64, (9,9), padding="same")(gen_ip)
  layers = PReLU(shared_axes=[1,2])(layers)

  temp = layers

  for i in range(num_res_block):
      layers = resnet_block(layers)

  layers = Conv2D(64, (3,3), padding="same")(layers)
  layers = BatchNormalization(momentum=0.5)(layers)
  layers = add([layers,temp])

  layers = upscale_block(layers)
  layers = upscale_block(layers)

  op = Conv2D(3, (9,9), padding="same")(layers)

  return Model(inputs=gen_ip, outputs=op, name='Generator')

In [ ]:
def discriminator_block(ip, filters, strides=1, bn=True):
  disc_model = Conv2D(filters, (3,3), strides = strides, padding="same")(ip)
  
  if bn:
      disc_model = BatchNormalization( momentum=0.8 )(disc_model)
  
  disc_model = LeakyReLU( alpha=0.2 )(disc_model)
  
  return disc_model

In [ ]:
def create_discriminator(disc_ip):
  df = 64
  
  d1 = discriminator_block(disc_ip, df, bn=False)
  d2 = discriminator_block(d1, df, strides=2)
  d3 = discriminator_block(d2, df*2)
  d4 = discriminator_block(d3, df*2, strides=2)
  d5 = discriminator_block(d4, df*4)
  d6 = discriminator_block(d5, df*4, strides=2)
  d7 = discriminator_block(d6, df*8)
  d8 = discriminator_block(d7, df*8, strides=2)

  d8_5 = Flatten()(d8)
  d9 = Dense(df*16)(d8_5)
  d10 = LeakyReLU(alpha=0.2)(d9)
  validity = Dense(1, activation='sigmoid')(d10)

  return Model(disc_ip, validity, name='Discriminator')

In [ ]:
def build_vgg(hr_shape):
  vgg = VGG19(weights="imagenet",include_top=False, input_shape=hr_shape)
  return Model(inputs=vgg.inputs, outputs=vgg.layers[10].output, name='VGG19')

In [ ]:
def create_comb(gen_model, disc_model, vgg, lr_ip, hr_ip):
  gen_img = gen_model(lr_ip)
  gen_features = vgg(gen_img)
  disc_model.trainable = False
  validity = disc_model(gen_img)
  
  return Model(inputs=[lr_ip, hr_ip], outputs=[validity, gen_features], name='Combine')

##Spliting Dataset Into Train & Test

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
lowres_train, lowres_test, highres_train, highres_test = train_test_split(X, y, test_size=0.2, random_state=42)

##Preparing Inputs

In [ ]:
highres_shape = (highres_train.shape[1], highres_train.shape[2], highres_train.shape[3])
lowres_shape = (lowres_train.shape[1], lowres_train.shape[2], lowres_train.shape[3])

In [ ]:
lowres_ip = Input(shape=lowres_shape)
highres_ip = Input(shape=highres_shape)

## Instantiating Generator

In [ ]:
generator = create_generator(lowres_ip, num_res_block = 16)
generator.summary()

##Visualizing Generator

In [ ]:
plot_model(generator, show_shapes=True, show_layer_names=True, dpi=64)

##Instantiating Discriminator

In [ ]:
discriminator = create_discriminator(highres_ip)
discriminator.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
discriminator.summary()

##Visualizing Discriminator

In [ ]:
plot_model(discriminator, show_shapes=True, show_layer_names=True, dpi=64)

##Instantiating VGG19

In [ ]:
vgg = build_vgg((256,256,3))
vgg.trainable = False
print(vgg.summary())

##Visualizing VGG19

In [ ]:
plot_model(vgg, show_shapes=True, show_layer_names=True, dpi=64)

##Building Combine Model i.e Attention U-Net

In [ ]:
gan_model = create_comb(generator, discriminator, vgg, lowres_ip, highres_ip)
gan_model.summary()

##Visualizing Combine Model i.e Attention U-Net

In [ ]:
plot_model(gan_model, show_shapes=True, show_layer_names=True, dpi=64)

##Compileing Model

In [ ]:
gan_model.compile(loss=["binary_crossentropy", "mse"], loss_weights=[1e-3, 1], optimizer="adam")

##Preparing Batch

In [ ]:
batch_size = 1  
train_lowres_batches = []
train_highres_batches = []
for it in range(int(highres_train.shape[0] / batch_size)):
  start_idx = it * batch_size
  end_idx = start_idx + batch_size
  train_highres_batches.append(highres_train[start_idx:end_idx])
  train_lowres_batches.append(lowres_train[start_idx:end_idx])

##Training The Model

In [ ]:
epochs = 30
per_epoch_g_losses = []
per_epoch_d_losses = []

for e in tqdm(range(epochs)):
  fake_label = np.zeros((batch_size, 1)) 
  real_label = np.ones((batch_size,1)) 
  
  g_losses = []
  d_losses = []

  for b in tqdm(range(len(train_highres_batches))):
    lr_imgs = train_lowres_batches[b] 
    hr_imgs = train_highres_batches[b] 
    
    fake_imgs = generator.predict_on_batch(lr_imgs) 
    
    #First, train the discriminator on fake and real HR images. 
    discriminator.trainable = True
    d_loss_gen = discriminator.train_on_batch(fake_imgs, fake_label)
    d_loss_real = discriminator.train_on_batch(hr_imgs, real_label)
    
    #Now, train the generator by fixing discriminator as non-trainable
    discriminator.trainable = False
    
    #Average the discriminator loss, just for reporting purposes. 
    d_loss = 0.5 * np.add(d_loss_gen, d_loss_real) 
    
    #Extract VGG features, to be used towards calculating loss
    image_features = vgg.predict(hr_imgs)
  
    #Train the generator via GAN. 
    #Remember that we have 2 losses, adversarial loss and content (VGG) loss
    g_loss, _, _ = gan_model.train_on_batch([lr_imgs, hr_imgs], [real_label, image_features])
    
    #Save losses to a list so we can average and report. 
    d_losses.append(d_loss)
    g_losses.append(g_loss)
      
  #Convert the list of losses to an array to make it easy to average    
  g_losses = np.array(g_losses)
  d_losses = np.array(d_losses)
  
  #Calculate the average losses for generator and discriminator
  g_loss = np.sum(g_losses, axis=0) / len(g_losses)
  d_loss = np.sum(d_losses, axis=0) / len(d_losses)

  per_epoch_g_losses.append(g_loss)
  per_epoch_d_losses.append(d_loss)
  
  #Report the progress during training. 
  print("epoch:", e+1 ,"g_loss:", g_loss, "d_loss:", d_loss)

  if (e+1) % 10 == 0:
    #Save the generator after every n epochs (Usually 10 epochs)
    generator.save("gen_e_"+ str(e+1) +".h5")

##Generator Loss Visualization

In [ ]:
plt.figure(figsize=(12,4))
plt.title("Generator Loss")
plt.plot(per_epoch_g_losses, ".:", label="Loss", linewidth=1)
plt.legend()
plt.show()

In [ ]:
d_loss = []
for value in tqdm(per_epoch_d_losses):
  d_loss.append(np.sum(value) / len(value))

##Discriminator Loss Visualization

In [ ]:
plt.figure(figsize=(12,4))
plt.title("Discriminator Loss")
plt.plot(d_loss, ".:", label="Loss", linewidth=1)
plt.legend()
plt.show()

##Attention U-Net Prediction

In [ ]:
from keras.models import load_model
from numpy.random import randint

In [ ]:
[X1, X2] = [lowres_test, highres_test]

plt.figure(figsize=(10,15))
id = randint(0, len(X1), 1)
n = 0
pred_out = None
for i in tqdm(range(12)):
  plt.subplot(4,3,i+1)
  
  if n==0:
    show_image((X1[id[0]] * 255).astype(np.uint8), title='Input Image')
    
    n+=1
  elif n==1:
    gen_image = generator.predict(X1[id])
    show_image((gen_image[0] * 255).astype(np.uint8), title='SuperResolution Image')
    
    n+=1
  elif n==2:
    show_image((X2[id[0]] * 255).astype(np.uint8), title='Original Image')  
    n=0
    id = randint(0, len(X1), 1)
plt.tight_layout()
plt.show()